<a href="https://colab.research.google.com/github/dabin3178/Korean-Newords-NLP/blob/main/0820_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/데청캠-심화프로젝트/data/labeling_oversampling.csv')
#df1 = pd.read_csv('/content/drive/MyDrive/Project/labeling.csv')
df1

,freq,freq_ratio,length,cohesion_forward,left_branching_entropy,right_branching_entropy,left_accessor_variety,starts_ratio,label
0,352,0.000307,3,0.124943,3.087665,2.561511,37,97,0
1,232,0.000202,3,0.055058,3.011088,0.948307,28,100,0
2,256,0.000223,4,0.136442,3.404558,2.785716,39,100,0
3,148,0.000129,4,0.170890,2.907327,2.910103,23,100,0
4,136,0.000119,3,0.070635,3.233183,2.799973,26,82,0
...,...,...,...,...,...,...,...,...,...
36615,140,0.000122,4,0.221532,1.579637,1.413289,9,100,1
36616,144,0.000125,4,0.940361,2.781463,2.651950,20,91,1
36617,394,0.000344,3,0.083799,3.733868,3.434715,54,100,1
36618,329,0.000287,2,0.460552,3.534511,3.402091,45,83,1


In [ ]:
df = df1.drop(['freq_ratio'], axis = 1)
df

,freq,length,cohesion_forward,left_branching_entropy,right_branching_entropy,left_accessor_variety,starts_ratio,label
0,352,3,0.124943,3.087665,2.561511,37,97,0
1,232,3,0.055058,3.011088,0.948307,28,100,0
2,256,4,0.136442,3.404558,2.785716,39,100,0
3,148,4,0.170890,2.907327,2.910103,23,100,0
4,136,3,0.070635,3.233183,2.799973,26,82,0
...,...,...,...,...,...,...,...,...
36615,140,4,0.221532,1.579637,1.413289,9,100,1
36616,144,4,0.940361,2.781463,2.651950,20,91,1
36617,394,3,0.083799,3.733868,3.434715,54,100,1
36618,329,2,0.460552,3.534511,3.402091,45,83,1


In [ ]:
from sklearn.model_selection import train_test_split

X = df[df.columns[0:7]].values #label 빼고 나머지 변수를 독립변수 X로.
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 1 # train : test = 70 : 30으로 하겠다, random_state는 train, test 뽑을 때마다 랜덤 ㄴㄴ 고정 시키겠다
)

In [ ]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

#모델의 정확도 확인
log.fit(X_train, y_train)
print('train 데이터셋의 정확도: %.2f' % log.score(X_train, y_train))
print('test 데이터셋의 정확도: %.2f' % log.score(X_test, y_test))

train 데이터셋의 정확도: 0.72
test 데이터셋의 정확도: 0.72


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
#더 자세한 평가 위해 classification_report모듈 사용
#모형 성능평가 위한 모듈. 정밀도(precision), 재현율(recall), F1-score, support를 구해줌
#https://sumniya.tistory.com/26
from sklearn.metrics import classification_report
y_pred = log.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.68      0.71      5516
           1       0.70      0.77      0.73      5470

    accuracy                           0.72     10986
   macro avg       0.72      0.72      0.72     10986
weighted avg       0.73      0.72      0.72     10986



In [ ]:
#모델의 회귀계수와 오즈비를 구해 독립변수가 분류 결정에 미치는 영향의 정도를 알아보기 위해 다른 방식의 로지스틱회귀분석 진행
import statsmodels.api as sm

X = sm.add_constant(X, has_constant='add')#상수항 추가
logit = sm.Logit(df[['label']],X)
result = logit.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Optimization terminated successfully.
         Current function value: 0.523233
         Iterations 7


In [ ]:
#이거 안 돌려도 됨 걍 내가 궁금해서 돌려본거임
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  label   No. Observations:                36620
Model:                          Logit   Df Residuals:                    36612
Method:                           MLE   Df Model:                            7
Date:                Fri, 20 Aug 2021   Pseudo R-squ.:                  0.2451
Time:                        07:33:20   Log-Likelihood:                -19161.
converged:                       True   LL-Null:                       -25383.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.2037      0.191    -37.788      0.000      -7.577      -6.830
x1            -0.0001   1.25e-05    -11.665      0.000      -0.000      -0.000
x2            -0.8245      0.019    -44.180      0.000      -0.861      -0.788
x3             4.0910      0.129     31.729      0.000       3.838       4.344
x4            -0.5350      0.023    -22.930      0.000      -0.581      -0.489
x5             0.9207      0.013     68.791      0.000       0.895       0.947
x6             0.0040      0.001      6.937      0.000       0.003       0.005
x7             0.0877      0.002     50.151      0.000       0.084       0.091
==============================================================================
"""

In [ ]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.245     
Dependent Variable: label            AIC:              38337.5936
Date:               2021-08-20 07:33 BIC:              38405.6604
No. Observations:   36620            Log-Likelihood:   -19161.   
Df Model:           7                LL-Null:          -25383.   
Df Residuals:       36612            LLR p-value:      0.0000    
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     7.0000                                       
-------------------------------------------------------------------
          Coef.    Std.Err.      z       P>|z|     [0.025    0.975]
-------------------------------------------------------------------
const    -7.2037     0.1906   -37.7881   0.0000   -7.5773   -6.8300
x1       -0.0001     0.0000   -11.6648   0.0000   -0.0002   -0.0001
x2       -0.8245     0.0187   -44.1804   0.0000   -0.8611   -0.7880
x3        4.0910     0.1289    31.7287   0.0000    3.8383    4.3437
x4       -0.5350     0.0233   -22.9302   0.0000   -0.5807   -0.4892
x5        0.9207     0.0134    68.7911   0.0000    0.8945    0.9470
x6        0.0040     0.0006     6.9371   0.0000    0.0028    0.0051
x7        0.0877     0.0017    50.1505   0.0000    0.0843    0.0911
=================================================================

"""

In [ ]:
#오즈비 구하기
#1에서부터 멀리 떨어질수록 종속변수와의 관계가 강함(큰 영향 줌)
#아무런 관계가 없을 때 오즈비는 1
np.exp(result.params)

const     0.000744
x1        0.999854
x2        0.438436
x3       59.798321
x4        0.585690
x5        2.511164
x6        1.003966
x7        1.091662
dtype: float64

위에서 구한 오즈비를 통해

x3, x5, x7, x6, x1, x4, x2 순으로 종속변수에 큰 영향을 미친다는 것을 알 수 있음

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)

print("Intercept: ", classifier.intercept_)
print("Parameter: ", classifier.coef_)

Intercept:  [-0.36898248]
Parameter:  [[-2.68069055e-04 -9.77640782e-01  2.07855153e-01 -1.10470509e+00
   7.75457309e-01  1.45088610e-02  4.70450854e-02]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


흠

In [ ]:
! pip install soynlp

     |████████████████████████████████| 416 kB 5.2 MB/s 


In [ ]:
import re

def text_preprocess(x): # 공백도 제외
    text=[]
    a = re.sub('[^가-힣\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

In [ ]:
n = int(input("몇 개의 문장을 입력하시겠습니까: "))
text = [""] * n
for i in range(n):
    text[i] = input("추가할 문장을 입력하세요: ")
    text[i] = (text[i] + '\n') * 64

몇 개의 문장을 입력하시겠습니까: 5
추가할 문장을 입력하세요: 이거 개꿀잼이다
추가할 문장을 입력하세요: 오늘 학교가다가 슈퍼카 봤다
추가할 문장을 입력하세요: 오늘 폰 떨어뜨려서 맴찢이였어
추가할 문장을 입력하세요: 갑자기 개가 달려와서 깜놀했어
추가할 문장을 입력하세요: 가지 너무 극혐이야


In [ ]:
! unzip -uq "/content/drive/MyDrive/Project/raw_data/raw_data.zip" -d /content/drive/MyDrive/Project/raw_data

In [ ]:
import pandas as pd

eta_secret_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/df_에타_sisa.csv")
eta_sisa_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/df_에타_sisa.csv")
eta_global_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/Globalfree.csv")
deoku_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/deoku.csv")
ppomppu_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/ppomppu1.csv")
ppomppu2_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/ppomppu2.csv")
youtube_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/유튜브15개.csv")
navernews_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/navernews.csv")
nate_pann_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/네이트판_통합.csv")
dc_inside_labeled = pd.read_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/raw_data/df_DC_total.csv")

labeled_data_df = pd.concat([eta_secret_labeled, eta_sisa_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, eta_global_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, deoku_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, ppomppu_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, ppomppu2_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, youtube_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, navernews_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, nate_pann_labeled],ignore_index=True)
labeled_data_df = pd.concat([labeled_data_df, dc_inside_labeled],ignore_index=True)

for column in labeled_data_df.columns:
    if column != 'comment':
        labeled_data_df = labeled_data_df.drop(column, axis=1)

labeled_data_df.to_csv("/content/drive/MyDrive/데청캠-심화프로젝트/data/total_data.csv", encoding='utf-8-sig', index=False)

labeled_data_df

,comment
0,문재인 국정 지지도 40% 신기하네....
1,민주당 뽑았던 내가 돌아선이유 상황이 이런데 맨날 변명 말돌리기 백신수급 문제없고 ...
2,이준석이 페미 여자표 다까먹는다고?? 전주 대비 3% 상승하여 민주당 역전. 어차...
3,홍은 안되는거 진짜 모르겠냐.. 시절인연이라는 것도 있듯이.. 되려면 직전에 됐어야...
4,솔직히 경선 시작되면 홍이 팍 치고 올라올 것 같음 지금은 야권에서 윤이 가장 압도...
...,...
1995730,야갤하는 여자는 도대체 어떤 인생을 사는 걸까
1995731,섹스
1995732,pp
1995733,흐읏흥


In [ ]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
import pandas as pd

df = labeled_data_df # pd.read_csv("/content/drive/MyDrive/Project/total_data.csv", encoding="utf-8", engine="python") # raw data 불러오기

train_corpus = "" # raw data를 쭉이을 빈 문자열

df['comment'] = df['comment'].apply(lambda x: str(x))
df['process'] = df['comment'].apply(lambda x: text_preprocess(x)) # 불필요한 문자 제거

for i in range(len(df['process'])):
    train_corpus += df['process'][i] + '\n' # 문자열을 쭉 이음.

for i in range(2):
    train_corpus += train_corpus + '\n'

In [ ]:
with open("/content/drive/MyDrive/데청캠-심화프로젝트/data/train_corpus.txt", 'w') as file: # txt 파일로 저장
    file.write(train_corpus)

In [ ]:
train_corpus = DoublespaceLineCorpus("/content/drive/MyDrive/데청캠-심화프로젝트/data/train_corpus.txt") # 저장한 txt 파일로 변환
len(train_corpus)

7982943

In [ ]:
# 코퍼스로 학습
word_extractor = WordExtractor()
word_extractor.train(train_corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.294 Gb
all cohesion probabilities was computed. # words = 530752
all branching entropies was computed # words = 1596511
all accessor variety was computed # words = 1596511


In [ ]:
n = int(input("몇 개의 단어를 입력하시겠습니까: "))
text = [""] * n
\ㅁ
for i in range(n):
    text[i] = input("기존 문장에 있던 단어를 입력하세요: ")

몇 개의 단어를 입력하시겠습니까: 13
기존 문장에 있던 단어를 입력하세요: 꿀잼
기존 문장에 있던 단어를 입력하세요: 가지
기존 문장에 있던 단어를 입력하세요: 너무
기존 문장에 있던 단어를 입력하세요: 극혐
기존 문장에 있던 단어를 입력하세요: 학교
기존 문장에 있던 단어를 입력하세요: 맴찢
기존 문장에 있던 단어를 입력하세요: 슈퍼카
기존 문장에 있던 단어를 입력하세요: 오늘
기존 문장에 있던 단어를 입력하세요: 가다가
기존 문장에 있던 단어를 입력하세요: 봤다
기존 문장에 있던 단어를 입력하세요: 했어
기존 문장에 있던 단어를 입력하세요: 이야
기존 문장에 있던 단어를 입력하세요: 이다


In [ ]:
tmp = text.copy()
text = []
for i in range(len(tmp)):
    try:
        word_score_table[tmp[i]].cohesion_forward
        text.append(tmp[i])
    except KeyError:
        pass

validation = {
    "words": text
}

validation_df = pd.DataFrame(validation)
validation_df['freq'] = validation_df['words'].apply(lambda x: word_score_table[x].leftside_frequency + word_score_table[x].rightside_frequency)
#validation_df['freq_ratio'] = validation_df['freq'].apply(lambda x: (x / sum(list(validation_df['freq']))) * 100)
validation_df['length'] = validation_df['words'].apply(lambda x: len(x))
validation_df['cohesion_forward'] = validation_df['words'].apply(lambda x: word_score_table[x].cohesion_forward)
validation_df['left_branching_entropy'] = validation_df['words'].apply(lambda x: word_score_table[x].left_branching_entropy)
validation_df['right_branching_entropy'] = validation_df['words'].apply(lambda x: word_score_table[x].right_branching_entropy)
validation_df['left_accessor_variety'] = validation_df['words'].apply(lambda x: word_score_table[x].left_accessor_variety)
validation_df['starts_ratio'] = validation_df['words'].apply(lambda x: word_score_table[x].leftside_frequency)

for i in range(len(validation_df['starts_ratio'])):
    validation_df['starts_ratio'][i] = (validation_df['starts_ratio'][i] / validation_df['freq'][i]) * 100

validation_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,words,freq,length,cohesion_forward,left_branching_entropy,right_branching_entropy,left_accessor_variety,starts_ratio
0,꿀잼,1276,2,0.211175,4.166356,3.603878,96,80
1,너무,57672,2,0.812609,4.643415,4.840593,642,96
2,극혐,864,2,0.096368,4.036975,2.973510,83,92
3,학교,9660,2,0.203434,4.182127,4.766682,287,52
4,맴찢,120,2,0.256410,3.122425,2.427353,24,100
5,슈퍼카,76,3,0.083578,2.625048,1.950409,15,100
6,오늘,74944,2,0.438947,5.266930,4.802966,767,98
7,봤다,1996,2,0.127631,4.046279,4.342866,99,57
8,했어,7460,2,0.092864,4.656190,4.816687,201,26


In [ ]:
features = validation_df[validation_df.columns[1:8]].values
features

array([[1.27600000e+03, 2.00000000e+00, 2.11175021e-01, 4.16635649e+00,
        3.60387785e+00, 9.60000000e+01, 8.00000000e+01],
       [5.76720000e+04, 2.00000000e+00, 8.12609253e-01, 4.64341513e+00,
        4.84059265e+00, 6.42000000e+02, 9.60000000e+01],
       [8.64000000e+02, 2.00000000e+00, 9.63680387e-02, 4.03697468e+00,
        2.97351001e+00, 8.30000000e+01, 9.20000000e+01],
       [9.66000000e+03, 2.00000000e+00, 2.03433754e-01, 4.18212662e+00,
        4.76668247e+00, 2.87000000e+02, 5.20000000e+01],
       [1.20000000e+02, 2.00000000e+00, 2.56410256e-01, 3.12242460e+00,
        2.42735273e+00, 2.40000000e+01, 1.00000000e+02],
       [7.60000000e+01, 3.00000000e+00, 8.35780720e-02, 2.62504816e+00,
        1.95040950e+00, 1.50000000e+01, 1.00000000e+02],
       [7.49440000e+04, 2.00000000e+00, 4.38947418e-01, 5.26693007e+00,
        4.80296563e+00, 7.67000000e+02, 9.80000000e+01],
       [1.99600000e+03, 2.00000000e+00, 1.27630990e-01, 4.04627910e+00,
        4.34286586e+00, 9

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(model.score(X_train, y_train))

0.7223219162050402


In [ ]:
print(model.coef_)

[[-2.68069055e-04 -9.77640782e-01  2.07855153e-01 -1.10470509e+00
   7.75457309e-01  1.45088610e-02  4.70450854e-02]]


In [ ]:
val_pred = model.predict(features)
print(val_pred)

[1 0 1 1 1 1 0 1 0]


In [ ]:
val_result = {
    "words": text,
    "result": val_pred
}

result_df = pd.DataFrame(val_result)
result_df

,words,result
0,꿀잼,1
1,너무,0
2,극혐,1
3,학교,1
4,맴찢,1
5,슈퍼카,1
6,오늘,0
7,봤다,1
8,했어,0


In [ ]:
print(model.score(X_test, y_test))

0.7228290551611142


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

In [ ]:
def get_confusion(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [ ]:
get_confusion(y_test, y_pred)

오차행렬:
 [[3726 1790]
 [1255 4215]]

정확도: 0.7228
정밀도: 0.7019
재현율: 0.7706
F1: 0.7346
AUC: 0.7230


In [ ]:
val_result = {
    "words": text,
    "result": val_pred
}

result_df = pd.DataFrame(val_result)
result_df

,words,result
0,꿀잼,1
1,너무,0
2,극혐,1
3,학교,1
4,맴찢,1
5,슈퍼카,1
6,오늘,0
7,봤다,1
8,했어,0
